In [18]:
import numpy as np
import matplotlib.pyplot as plt
from ansys.mapdl.core import launch_mapdl
mapdl=launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise Academic Student
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [19]:
# Takes in Pressure, Crack length and Thickness
def calculateK (press,a,t):
    mapdl.clear()
    mapdl.prep7()
    
    # Define Material Property
    E = 210e9
    nu = 0.3
    YS = 240e6
    # Defining Geometry 
    # Corners of the rectangle
    m0 = mapdl.k("", 0, -0.1)
    m1 = mapdl.k("", 0,0.1002)
    m2 = mapdl.k("",0.005,0.1002)
    m3 = mapdl.k("",0.005,0.1)
    m4 = mapdl.k("", 0.1, 0.1)
    m5 = mapdl.k("", 0.1, -0.1)
    main = mapdl.a(m0,m1,m2,m3,m4,m5)
    # Crack geometry
    k0 = mapdl.k("",0,-0.0001)
    k1 = mapdl.k("",0.1*a,-0.0001)
    k2 = mapdl.k("",a,0)
    k3 = mapdl.k("",0.1*a,0.0001)
    k4 = mapdl.k("",0,0.0001)
    crack = mapdl.a(k0, k1, k2,k3,k4)
    # Substract the areas and extrude with Thickness t
    aout = mapdl.asba(main, crack)
    mapdl.vext(aout, dz=t)
    # mapdl.vplot(show_lines=True, quality=5)
    mapdl.et(1,"SOLID285")
    # Define the Mesh
    mapdl.smrtsize(2)
    mapdl.aesize("ALL", 0.003)
    mapdl.vmesh(1)
    scon1 = mapdl.k(10000,0.1,0,0)
    scon2 = mapdl.k(10001,0.2,0,0)
    
    # mapdl.eplot(vtk=True, show_edges=True, show_axes=False, line_width=2, background="w")
    
    # Using the SI units
    mapdl.units("SI")  # SI - International system (m, kg, s, K).

    # Define a material (nominal steel in SI)
    mapdl.mp("EX", 1, E)  # Elastic moduli in Pa (kg/(m*s**2))
    mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
    mapdl.mp("NUXY", 1, nu)  # Poisson's Ratio

    # Fix the lower side.
    mapdl.nsel("S", "LOC", "Y", -0.1)
    mapdl.d("ALL", "UY")
    mapdl.d("ALL", "UZ")
    mapdl.d("ALL", "UX")

    mapdl.nsel("S", "LOC", "Y", 0.1)
    mapdl.cp(5, "UX", "ALL")

    mapdl.asel("S",'LOC',"Y",0.1002)
    mapdl.sfa('ALL', '', 'PRES', -press)
    mapdl.allsel(mute=True)
    mapdl.run("/SOLU")
    mapdl.antype("STATIC")
    mapdl.solve()
    mapdl.finish(mute=True)
    mapdl.post1()
    mapdl.path('P',3,5)
    node1 = mapdl.queries.node(a,0,0)
    node2 = mapdl.queries.node(0.5*a,0.0001,0)
    node3 = mapdl.queries.node(0,0.0001,0)
    mapdl.ppath(1, node1)
    mapdl.ppath(2, node2)
    mapdl.ppath(3, node3)
    mapdl.local(11,0,a,0,0)
    mapdl.csys(11)
    a = mapdl.kcalc(0,1,0,0)
    result = mapdl.result
    #result.plot_principal_nodal_stress(
    #    0,
    #    "SEQV",
    #    lighting=False,
    #    background="w",
    #    show_edges=True,
    #    text_color="k",
    #    add_text=False,
    #    )
    words = a.split(' ')
    
    # Extracting the K1 value from the String output
    for i in range(len(words)):
        if words[i]=='KI':
            x=i
    K1 = words[x+3][:-1] 
    print(f"K1 is equal to {K1}")
    return K1

In [ ]:
P = [50e6,100e6,150e6,200e6,300e6,350e6,400e6,450e6,500e6,550e6,600e6,650e6,700e6,750e6,800e6,850e6,900e6,950e6,1000e6]
#A = np.arange(0.01,0.015,0.001) 
A = [0.009,0.010, 0.011,0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.020]
#T = np.arange(0.004,0.012,0.001)
T = [0.004, 0.005, 0.006,0.007,0.008,0.009,0.010, 0.011,0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.020]
data = np.zeros((len(P)*len(A)*len(T),4))
count =0
for i in range(len(P)):
    for j in range(len(A)):
        for k in range(len(T)):
            print(f"{P[i]}; {A[j]}; {T[k]}")
            K1 = calculateK(P[i],A[j],T[k])
            Area = 0.005*A[j]
            Force = Area*P[i]
            data[count,0] = Force
            data[count,1] = A[j]
            data[count,2] = T[k]
            data[count,3] = K1
            count=count+1    
      
            
        

50000000.0; 0.009; 0.004
K1 is equal to 0.85506E+07
50000000.0; 0.009; 0.005
K1 is equal to 0.78989E+07
50000000.0; 0.009; 0.006
K1 is equal to 0.85507E+07
50000000.0; 0.009; 0.007
K1 is equal to 0.85164E+07
50000000.0; 0.009; 0.008
K1 is equal to 0.83809E+07
50000000.0; 0.009; 0.009
K1 is equal to 0.81249E+07
50000000.0; 0.009; 0.01
K1 is equal to 0.80646E+07
50000000.0; 0.009; 0.011
K1 is equal to 0.87851E+07
50000000.0; 0.009; 0.012
K1 is equal to 0.82833E+07
50000000.0; 0.009; 0.013
K1 is equal to 0.86963E+07
50000000.0; 0.009; 0.014
K1 is equal to 0.84946E+07
50000000.0; 0.009; 0.015
K1 is equal to 0.80171E+07
50000000.0; 0.009; 0.016
K1 is equal to 0.84926E+07
50000000.0; 0.009; 0.017
K1 is equal to 0.87588E+07
50000000.0; 0.009; 0.018
K1 is equal to 0.82939E+07
50000000.0; 0.009; 0.019
K1 is equal to 0.87597E+07
50000000.0; 0.009; 0.02
K1 is equal to 0.81488E+07
50000000.0; 0.01; 0.004
K1 is equal to 0.93139E+07
50000000.0; 0.01; 0.005
K1 is equal to 0.90126E+07
50000000.0; 0.01

K1 is equal to 0.15162E+08
50000000.0; 0.018; 0.01
K1 is equal to 0.15823E+08
50000000.0; 0.018; 0.011
K1 is equal to 0.15282E+08
50000000.0; 0.018; 0.012
K1 is equal to 0.15602E+08
50000000.0; 0.018; 0.013
K1 is equal to 0.15229E+08
50000000.0; 0.018; 0.014
K1 is equal to 0.15469E+08
50000000.0; 0.018; 0.015
K1 is equal to 0.15167E+08
50000000.0; 0.018; 0.016
K1 is equal to 0.15506E+08
50000000.0; 0.018; 0.017
K1 is equal to 0.15521E+08
50000000.0; 0.018; 0.018
K1 is equal to 0.15143E+08
50000000.0; 0.018; 0.019
K1 is equal to 0.15319E+08
50000000.0; 0.018; 0.02
K1 is equal to 0.14781E+08
50000000.0; 0.019; 0.004
K1 is equal to 0.15733E+08
50000000.0; 0.019; 0.005
K1 is equal to 0.16276E+08
50000000.0; 0.019; 0.006
K1 is equal to 0.15043E+08
50000000.0; 0.019; 0.007
K1 is equal to 0.16275E+08
50000000.0; 0.019; 0.008
K1 is equal to 0.16330E+08
50000000.0; 0.019; 0.009
K1 is equal to 0.15813E+08
50000000.0; 0.019; 0.01
K1 is equal to 0.15236E+08
50000000.0; 0.019; 0.011
K1 is equal to 

K1 is equal to 0.26181E+08
100000000.0; 0.015; 0.014
K1 is equal to 0.25660E+08
100000000.0; 0.015; 0.015
K1 is equal to 0.26328E+08
100000000.0; 0.015; 0.016
K1 is equal to 0.26584E+08
100000000.0; 0.015; 0.017
K1 is equal to 0.26462E+08
100000000.0; 0.015; 0.018
K1 is equal to 0.26463E+08
100000000.0; 0.015; 0.019
K1 is equal to 0.26251E+08
100000000.0; 0.015; 0.02
K1 is equal to 0.25445E+08
100000000.0; 0.016; 0.004
K1 is equal to 0.27425E+08
100000000.0; 0.016; 0.005
K1 is equal to 0.26204E+08
100000000.0; 0.016; 0.006
K1 is equal to 0.26596E+08
100000000.0; 0.016; 0.007
K1 is equal to 0.26055E+08
100000000.0; 0.016; 0.008
K1 is equal to 0.26759E+08
100000000.0; 0.016; 0.009
K1 is equal to 0.26979E+08
100000000.0; 0.016; 0.01
K1 is equal to 0.27067E+08
100000000.0; 0.016; 0.011
K1 is equal to 0.27503E+08
100000000.0; 0.016; 0.012
K1 is equal to 0.27349E+08
100000000.0; 0.016; 0.013
K1 is equal to 0.26006E+08
100000000.0; 0.016; 0.014
K1 is equal to 0.26532E+08
100000000.0; 0.016; 0

K1 is equal to 0.31785E+08
150000000.0; 0.012; 0.017
K1 is equal to 0.31337E+08
150000000.0; 0.012; 0.018
K1 is equal to 0.30597E+08
150000000.0; 0.012; 0.019
K1 is equal to 0.31786E+08
150000000.0; 0.012; 0.02
K1 is equal to 0.31957E+08
150000000.0; 0.013; 0.004
K1 is equal to 0.35214E+08
150000000.0; 0.013; 0.005
K1 is equal to 0.35370E+08
150000000.0; 0.013; 0.006
K1 is equal to 0.34922E+08
150000000.0; 0.013; 0.007
K1 is equal to 0.35652E+08
150000000.0; 0.013; 0.008
K1 is equal to 0.33684E+08
150000000.0; 0.013; 0.009
K1 is equal to 0.34702E+08
150000000.0; 0.013; 0.01
K1 is equal to 0.34583E+08
150000000.0; 0.013; 0.011
K1 is equal to 0.33780E+08
150000000.0; 0.013; 0.012
K1 is equal to 0.34312E+08
150000000.0; 0.013; 0.013
K1 is equal to 0.34401E+08
150000000.0; 0.013; 0.014
K1 is equal to 0.34672E+08
150000000.0; 0.013; 0.015
K1 is equal to 0.34622E+08
150000000.0; 0.013; 0.016
K1 is equal to 0.33941E+08
150000000.0; 0.013; 0.017
K1 is equal to 0.32790E+08
150000000.0; 0.013; 0

K1 is equal to 0.35039E+08
200000000.0; 0.009; 0.02
K1 is equal to 0.32595E+08
200000000.0; 0.01; 0.004
K1 is equal to 0.37256E+08
200000000.0; 0.01; 0.005
K1 is equal to 0.36051E+08
200000000.0; 0.01; 0.006
K1 is equal to 0.36795E+08
200000000.0; 0.01; 0.007
K1 is equal to 0.34855E+08
200000000.0; 0.01; 0.008
K1 is equal to 0.37053E+08
200000000.0; 0.01; 0.009
K1 is equal to 0.36146E+08
200000000.0; 0.01; 0.01
K1 is equal to 0.34960E+08
200000000.0; 0.01; 0.011
K1 is equal to 0.36318E+08
200000000.0; 0.01; 0.012
K1 is equal to 0.34526E+08
200000000.0; 0.01; 0.013
K1 is equal to 0.36874E+08
200000000.0; 0.01; 0.014
K1 is equal to 0.36533E+08
200000000.0; 0.01; 0.015
K1 is equal to 0.33771E+08
200000000.0; 0.01; 0.016
K1 is equal to 0.35234E+08
200000000.0; 0.01; 0.017
K1 is equal to 0.35742E+08
200000000.0; 0.01; 0.018
K1 is equal to 0.36518E+08
200000000.0; 0.01; 0.019
K1 is equal to 0.34519E+08
200000000.0; 0.01; 0.02
K1 is equal to 0.34597E+08
200000000.0; 0.011; 0.004
K1 is equal t

...

K1 is equal to 0.73065E+08
400000000.0; 0.01; 0.015
K1 is equal to 0.67543E+08
400000000.0; 0.01; 0.016
K1 is equal to 0.70468E+08
400000000.0; 0.01; 0.017
K1 is equal to 0.71484E+08
400000000.0; 0.01; 0.018
K1 is equal to 0.73036E+08
400000000.0; 0.01; 0.019
K1 is equal to 0.69037E+08
400000000.0; 0.01; 0.02
K1 is equal to 0.69195E+08
400000000.0; 0.011; 0.004
K1 is equal to 0.84078E+08
400000000.0; 0.011; 0.005
K1 is equal to 0.80909E+08
400000000.0; 0.011; 0.006
K1 is equal to 0.76756E+08
400000000.0; 0.011; 0.007
K1 is equal to 0.81180E+08
400000000.0; 0.011; 0.008
K1 is equal to 0.77989E+08
400000000.0; 0.011; 0.009
K1 is equal to 0.80164E+08
400000000.0; 0.011; 0.01
K1 is equal to 0.78768E+08
400000000.0; 0.011; 0.011
K1 is equal to 0.81320E+08
400000000.0; 0.011; 0.012
K1 is equal to 0.78734E+08
400000000.0; 0.011; 0.013
K1 is equal to 0.79482E+08
400000000.0; 0.011; 0.014
K1 is equal to 0.77234E+08
400000000.0; 0.011; 0.015
K1 is equal to 0.77939E+08
400000000.0; 0.011; 0.016
K

K1 is equal to 0.12988E+09
400000000.0; 0.019; 0.018
K1 is equal to 0.13166E+09
400000000.0; 0.019; 0.019
K1 is equal to 0.12553E+09
400000000.0; 0.019; 0.02
K1 is equal to 0.12823E+09
400000000.0; 0.02; 0.004
K1 is equal to 0.14147E+09
400000000.0; 0.02; 0.005
K1 is equal to 0.13898E+09
400000000.0; 0.02; 0.006
K1 is equal to 0.14101E+09
400000000.0; 0.02; 0.007
K1 is equal to 0.13635E+09
400000000.0; 0.02; 0.008
K1 is equal to 0.13915E+09
400000000.0; 0.02; 0.009
K1 is equal to 0.13942E+09
400000000.0; 0.02; 0.01
K1 is equal to 0.13757E+09
400000000.0; 0.02; 0.011
K1 is equal to 0.13529E+09
400000000.0; 0.02; 0.012
K1 is equal to 0.13759E+09
400000000.0; 0.02; 0.013
K1 is equal to 0.13857E+09
400000000.0; 0.02; 0.014
K1 is equal to 0.13863E+09
400000000.0; 0.02; 0.015
K1 is equal to 0.13902E+09
400000000.0; 0.02; 0.016
K1 is equal to 0.13417E+09
400000000.0; 0.02; 0.017
K1 is equal to 0.13650E+09
400000000.0; 0.02; 0.018
K1 is equal to 0.14037E+09
400000000.0; 0.02; 0.019
K1 is equal

K1 is equal to 0.12107E+09
450000000.0; 0.017; 0.004
K1 is equal to 0.13779E+09
450000000.0; 0.017; 0.005
K1 is equal to 0.13392E+09
450000000.0; 0.017; 0.006
K1 is equal to 0.13153E+09
450000000.0; 0.017; 0.007
K1 is equal to 0.13044E+09
450000000.0; 0.017; 0.008
K1 is equal to 0.12936E+09
450000000.0; 0.017; 0.009
K1 is equal to 0.13372E+09
450000000.0; 0.017; 0.01
K1 is equal to 0.13372E+09
450000000.0; 0.017; 0.011
K1 is equal to 0.12839E+09
450000000.0; 0.017; 0.012
K1 is equal to 0.12767E+09
450000000.0; 0.017; 0.013
K1 is equal to 0.13491E+09
450000000.0; 0.017; 0.014
K1 is equal to 0.13190E+09
450000000.0; 0.017; 0.015
K1 is equal to 0.13103E+09
450000000.0; 0.017; 0.016
K1 is equal to 0.12759E+09
450000000.0; 0.017; 0.017
K1 is equal to 0.13349E+09
450000000.0; 0.017; 0.018
K1 is equal to 0.13037E+09
450000000.0; 0.017; 0.019
K1 is equal to 0.12964E+09
450000000.0; 0.017; 0.02
K1 is equal to 0.13092E+09
450000000.0; 0.018; 0.004
K1 is equal to 0.13666E+09
450000000.0; 0.018; 0

K1 is equal to 0.12151E+09
500000000.0; 0.014; 0.007
K1 is equal to 0.12312E+09
500000000.0; 0.014; 0.008
K1 is equal to 0.12292E+09
500000000.0; 0.014; 0.009
K1 is equal to 0.11895E+09
500000000.0; 0.014; 0.01
K1 is equal to 0.12488E+09
500000000.0; 0.014; 0.011
K1 is equal to 0.12207E+09
500000000.0; 0.014; 0.012
K1 is equal to 0.11973E+09
500000000.0; 0.014; 0.013
K1 is equal to 0.12029E+09
500000000.0; 0.014; 0.014
K1 is equal to 0.12019E+09
500000000.0; 0.014; 0.015
K1 is equal to 0.12057E+09
500000000.0; 0.014; 0.016
K1 is equal to 0.11769E+09
500000000.0; 0.014; 0.017
K1 is equal to 0.12330E+09
500000000.0; 0.014; 0.018
K1 is equal to 0.12244E+09
500000000.0; 0.014; 0.019
K1 is equal to 0.12015E+09
500000000.0; 0.014; 0.02
K1 is equal to 0.12102E+09
500000000.0; 0.015; 0.004
K1 is equal to 0.13081E+09
500000000.0; 0.015; 0.005
K1 is equal to 0.12864E+09
500000000.0; 0.015; 0.006
K1 is equal to 0.12953E+09
500000000.0; 0.015; 0.007
K1 is equal to 0.12928E+09
500000000.0; 0.015; 0

K1 is equal to 0.11023E+09
550000000.0; 0.011; 0.01
K1 is equal to 0.10831E+09
550000000.0; 0.011; 0.011
K1 is equal to 0.11182E+09
550000000.0; 0.011; 0.012
K1 is equal to 0.10826E+09
550000000.0; 0.011; 0.013
K1 is equal to 0.10929E+09
550000000.0; 0.011; 0.014
K1 is equal to 0.10620E+09
550000000.0; 0.011; 0.015
K1 is equal to 0.10717E+09
550000000.0; 0.011; 0.016
K1 is equal to 0.10660E+09
550000000.0; 0.011; 0.017
K1 is equal to 0.11098E+09
550000000.0; 0.011; 0.018
K1 is equal to 0.11184E+09
550000000.0; 0.011; 0.019
K1 is equal to 0.10502E+09
550000000.0; 0.011; 0.02
K1 is equal to 0.10951E+09
550000000.0; 0.012; 0.004
K1 is equal to 0.12064E+09
550000000.0; 0.012; 0.005
K1 is equal to 0.11633E+09
550000000.0; 0.012; 0.006
K1 is equal to 0.12022E+09
550000000.0; 0.012; 0.007
K1 is equal to 0.11691E+09
550000000.0; 0.012; 0.008
K1 is equal to 0.11119E+09
550000000.0; 0.012; 0.009
K1 is equal to 0.12048E+09
550000000.0; 0.012; 0.01
K1 is equal to 0.12322E+09
550000000.0; 0.012; 0.

In [10]:
data[10:20,:]

array([[5.0000e+07, 1.1000e-02, 6.0000e-03, 1.0545e+07],
       [5.0000e+07, 1.1000e-02, 7.0000e-03, 1.0222e+07],
       [5.0000e+07, 1.2000e-02, 4.0000e-03, 1.1182e+07],
       [5.0000e+07, 1.2000e-02, 5.0000e-03, 1.1207e+07],
       [5.0000e+07, 1.2000e-02, 6.0000e-03, 1.1124e+07],
       [5.0000e+07, 1.2000e-02, 7.0000e-03, 1.0996e+07],
       [1.0000e+08, 9.0000e-03, 4.0000e-03, 1.7790e+07],
       [1.0000e+08, 9.0000e-03, 5.0000e-03, 1.7697e+07],
       [1.0000e+08, 9.0000e-03, 6.0000e-03, 1.6811e+07],
       [1.0000e+08, 9.0000e-03, 7.0000e-03, 1.6906e+07]])

In [ ]:
len(data)

In [8]:
a =len(P)*len(A)*len(T)
a

1309

In [60]:
np.save('data54.npy',data)

In [ ]:
P = [50e6,100e6,150e6,200e6]
A = [0.009,0.010, 0.011]
T = [0.004, 0.005, 0.006]